In [ ]:
!pip install sagemaker >> /dev/null
%store -r 

In [ ]:
ACCESS_KEY = image_sgmkr_config['ACCESS_KEY']
SECRET_KEY = image_sgmkr_config['SECRET_KEY']
BUCKET = image_sgmkr_config['BUCKET']
PREFIX = image_sgmkr_config['PREFIX']
REGION_NAME = image_sgmkr_config['REGION_NAME']
ROLE = image_sgmkr_config['ROLE']
MONITOR_NAME = image_sgmkr_config["MONITOR_NAME"]

In [ ]:
import os
os.environ["AWS_ACCESS_KEY_ID"] = ACCESS_KEY
os.environ["AWS_SECRET_ACCESS_KEY"] = SECRET_KEY
os.environ["AWS_DEFAULT_REGION"] = REGION_NAME
HOME = os.getenv("HOME")
if HOME:
    EXECUTABLE_DIR = os.path.join(HOME,".local", "bin")
    PATH = os.getenv("PATH")
    if EXECUTABLE_DIR not in PATH:
        os.environ["PATH"] = f"{PATH}:{EXECUTABLE_DIR}"
    PATH = os.getenv("PATH")
    
os.environ["AWS_ACCESS_KEY_ID"] = ACCESS_KEY
os.environ["AWS_SECRET_ACCESS_KEY"] = SECRET_KEY
os.environ["AWS_DEFAULT_REGION"] = REGION_NAME

In [ ]:
import tensorflow as tf
import os
import boto3
from sagemaker import get_execution_role
from sagemaker import Session
import sagemaker
import numpy as np

In [ ]:
role = ROLE
sess = Session()
bucket = BUCKET

In [ ]:
model_archive = 'model.tar.gz'
model_data = sess.upload_data(path=model_archive,bucket=bucket, key_prefix='model')

In [ ]:
data_capture_prefix = "{}/datacapture".format(PREFIX)
s3_capture_upload_path = "s3://{}/{}".format(BUCKET, data_capture_prefix)
print(s3_capture_upload_path)

In [ ]:
ENDPOINT_NAME = MONITOR_NAME
image_sgmkr_config['ENDPOINT_NAME'] = ENDPOINT_NAME
%store image_sgmkr_config

In [ ]:
from sagemaker.tensorflow.serving import Model
from sagemaker.model_monitor import DataCaptureConfig

instance_type = "ml.t2.medium"
tf_framework_version = "2.0"

sm_model = Model(model_data=model_data, 
                 framework_version=tf_framework_version,role=role)

data_capture_config = DataCaptureConfig(enable_capture=True, sampling_percentage=100, 
                                        destination_s3_uri=s3_capture_upload_path)

uncompiled_predictor = sm_model.deploy(initial_instance_count=1, 
                                       instance_type=instance_type,
                                       endpoint_name = ENDPOINT_NAME,
                                       data_capture_config=data_capture_config,
                                       wait=False)
print("Sagemaker will take 10-15 mins to create the endpoint")

In [ ]:
CLEANUP = False
if CLEANUP:
    client = boto3.client(service_name="sagemaker", aws_access_key_id=ACCESS_KEY,
                                     aws_secret_access_key=SECRET_KEY,
                                     region_name=REGION_NAME)
    client.delete_endpoint(EndpointName = ENDPOINT_NAME)
    client.delete_endpoint_config(EndpointConfigName = ENDPOINT_NAME)
    client.delete_model(ModelName=ENDPOINT_NAME)